<a href="https://colab.research.google.com/github/ark-017/Deep-learning-assignment/blob/main/prob3e.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Import Libraries

In [1]:
import torch
import torch.nn as nn
import torchvision
import torchvision.transforms as transforms
from torch.utils.data import DataLoader

# Set device
device = torch.device("cpu")

## *Load and transform CIFAR-10 dataset*

In [2]:
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

train_dataset = torchvision.datasets.CIFAR10(root='./data', train=True, transform=transform, download=True)
test_dataset = torchvision.datasets.CIFAR10(root='./data', train=False, transform=transform, download=True)

train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False)

100%|██████████| 170M/170M [00:02<00:00, 66.5MB/s]


### *Defining a custom CNN model*

In [5]:
class EasyCNN(nn.Module):
    def __init__(self):
        super(EasyCNN, self).__init__()
        self.conv_layers = nn.Sequential(
            nn.Conv2d(3, 32, kernel_size=3, padding=1),  # Output: 32x32x32
            nn.ReLU(),
            nn.MaxPool2d(2, 2),                          # Output: 32x16x16

            nn.Conv2d(32, 64, kernel_size=3, padding=1), # Output: 64x16x16
            nn.ReLU(),
            nn.MaxPool2d(2, 2),                          # Output: 64x8x8

            nn.Conv2d(64, 128, kernel_size=3, padding=1),# Output: 128x8x8
            nn.ReLU(),
            nn.MaxPool2d(2, 2),                          # Output: 128x4x4
        )
        self.fc_layers = nn.Sequential(
            nn.Flatten(),
            nn.Linear(128 * 4 * 4, 512),
            nn.ReLU(),
            nn.Dropout(0.5),
            nn.Linear(512, 10)
        )

    def forward(self, x):
        x = self.conv_layers(x)
        x = self.fc_layers(x)
        return x


## *Instantiate model, loss, optimizer*

In [6]:
model = EasyCNN().to(device)
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

 ## *Train Model*

In [9]:
num_epochs = 1
for epoch in range(num_epochs):
    model.train()
    total_loss = 0
    for i, (images, labels) in enumerate(train_loader):

        print(f'Batch {i+1}/{len(train_loader)}')

        images, labels = images.to(device), labels.to(device)

        outputs = model(images)
        loss = criterion(outputs, labels)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        total_loss += loss.item()

    print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {total_loss/len(train_loader):.4f}")

Batch 1/782
Batch 2/782
Batch 3/782
Batch 4/782
Batch 5/782
Batch 6/782
Batch 7/782
Batch 8/782
Batch 9/782
Batch 10/782
Batch 11/782
Batch 12/782
Batch 13/782
Batch 14/782
Batch 15/782
Batch 16/782
Batch 17/782
Batch 18/782
Batch 19/782
Batch 20/782
Batch 21/782
Batch 22/782
Batch 23/782
Batch 24/782
Batch 25/782
Batch 26/782
Batch 27/782
Batch 28/782
Batch 29/782
Batch 30/782
Batch 31/782
Batch 32/782
Batch 33/782
Batch 34/782
Batch 35/782
Batch 36/782
Batch 37/782
Batch 38/782
Batch 39/782
Batch 40/782
Batch 41/782
Batch 42/782
Batch 43/782
Batch 44/782
Batch 45/782
Batch 46/782
Batch 47/782
Batch 48/782
Batch 49/782
Batch 50/782
Batch 51/782
Batch 52/782
Batch 53/782
Batch 54/782
Batch 55/782
Batch 56/782
Batch 57/782
Batch 58/782
Batch 59/782
Batch 60/782
Batch 61/782
Batch 62/782
Batch 63/782
Batch 64/782
Batch 65/782
Batch 66/782
Batch 67/782
Batch 68/782
Batch 69/782
Batch 70/782
Batch 71/782
Batch 72/782
Batch 73/782
Batch 74/782
Batch 75/782
Batch 76/782
Batch 77/782
Batch 78

## *Evalution of Model*

In [10]:
model.eval()
correct = 0
total = 0
with torch.no_grad():
    for images, labels in test_loader:
        images, labels = images.to(device), labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print(f'Accuracy of custom CNN on CIFAR-10: {100 * correct / total:.2f}%')

Accuracy of custom CNN on CIFAR-10: 59.90%
